In [1]:
import os
import glob
import numpy as np
import nibabel as nib
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, permutation_test_score
from nilearn.input_data import NiftiMasker 
from nilearn.image import concat_imgs, index_img, smooth_img

/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# ---STEP 1---
### Concatenate the imagine data into a NIFTI-2 file. 
Note: the data does not fit into a NIFTI-1 file, due to large number of subs. 

In [2]:
def getData():
    global basepath
#set basepath
    #basepath=os.path.join('/projects','niblab','data','eric_data','W1','imagine')
    basepath= '//Users/nikkibytes/Documents/testing/ChocoData'
#make a list of the files to concat
#we are concatinating all the 'filtered_func_data.nii.gz' files within the directories
    #all_func = glob.glob(os.path.join(basepath,'level1_grace_edit','cs*++.feat','filtered_func_data.nii.gz'))
    all_func = glob.glob(os.path.join(basepath, 'cs*++.feat', 'filtered_func_data.nii.gz'))

    return all_func

In [3]:
def convertToNifti2():
#load in all the files from the glob above, then convert them from nifti1 to nifti2
    ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in all_func)
#concat, this is with nibabel, but should work with nilearn too
    ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
#set the output file name
    outfile=os.path.join(basepath,'concatenated_imagine.nii')
#write the file
    ni2_concat.to_filename(outfile)

# ---STEP 2---
### Load & prepare MRI data

In [4]:
def prepData():
    priint("We are now loading and preparing MRI data......")
    #load, fxnl, anatomical, & mask for plotting
    fmri_subjs=os.path.join(basepath, 'concatenated_imagine.nii')
    
    ## -- What are these?    
    average_ana=os.path.join(basepath,'CS_avg_mprage_image.nii.gz') 
    imag_mask=os.path.join('/projects','niblab','scripts','nilean_stuff','power_roimask_4bi.nii.gz')

    #plot mask (Power ROIs) over anatomical tlhat is defined above
    #plotting.plot_roi(imag_mask,bg_img=average_ana,cmap='Paired')

    #load labels for the functional data
    stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_67_sub.csv')
    labels = np.recfromcsv(stim, delimiter=",")
    print(labels)
#Its shape corresponds to the number of time-points times the number of voxels in the mask

#Retrieve the behavioral targets, that we are going to predict in the decoding
    y_mask = labels['labels']
    subs = labels['subs']
    return y_mask, subs

# ---STEP 3---
### Feature selection
To keep only data corresponding to app food or unapp food, we create a mask of the samples belonging to the condition.


In [50]:
def featureSelection(y_mask):
    condition_mask = np.logical_or(y_mask == b'app',y_mask == b'unapp')
    print(condition_mask.shape)
    y = y_mask[condition_mask]
    print(y)
    n_conditions = np.size(np.unique(y))

#prepare the fxnl data. 
    nifti_masker = NiftiMasker(mask_img=imag_mask,
                           smoothing_fwhm=4,standardize=True,
                           memory="nilearn_cache",memory_level=1)

    fmri_trans = nifti_masker.fit_transform(fmri_subjs)
    print(fmri_trans)
    X = fmri_trans[condition_mask]
    subs = subs[condition_mask]

# ---STEP 4---
###  Setting prediction  & testing the classifer

In [51]:
def setPrediction():

#setting prediction  & testing the classifer
    svc = SVC(kernel='linear')
    print(svc)

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test, namely Anova. We set the number of features to be selected to 500
    feature_selection = SelectKBest(f_classif, k=3000)

# We have our classifier (SVC), our feature selection (SelectKBest), and now, we can plug them together in a *pipeline* that performs the two operations successively:
    anova_svc = Pipeline([('anova',feature_selection), ('svc',svc)])

#fit the decoder and predict
    anova_svc.fit(X, y)
    y_pred = anova_svc.predict(X)

    cv = LeaveOneLabelOut(subs[subs < 1])

    k_range = [10, 15, 30, 50, 150, 300, 500, 1000, 1500, 3000, 5000]
    cv_scores = []
    scores_validation = []

# we are working with a composite estimator: 
# a pipeline of feature selection followed by SVC. Thus to give the name of the parameter that we want to tune we need to give the name of the step in
# the pipeline, followed by the name of the parameter, with ‘__’ as a separator.
# We are going to tune the parameter 'k' of the step called 'anova' in the pipeline. Thus we need to address it as 'anova__k'.
# Note that GridSearchCV takes an n_jobs argument that can make it go much faster
    grid = GridSearchCV(anova_svc, param_grid={'anova__k': k_range},n_jobs=-1)
    nested_cv_scores = cross_val_score(grid, X, y)
    classification_accuracy = np.mean(nested_cv_scores)
    print("Classification accuracy: %.4f / Chance level: %f" %
      (classification_accuracy, 1. / n_conditions))
    
    for k in k_range:
        feature_selection.k = k
        cv_scores.append(np.mean(
        cross_val_score(anova_svc, X[subs == 1], y[subs == 1])))
        print("CV score: %.4f" % cv_scores[-1])

        anova_svc.fit(X[subs == 1], y[subs == 1])
        y_pred = anova_svc.predict(X[subs == 0])
        scores_validation.append(np.mean(y_pred == y[subs == 0]))
        print("score validation: %.4f" % scores_validation[-1])

# ---STEP 5---
### flipping the martix backinto an image

In [32]:
def flipImage():
    coef = svc.coef_
    print(coef)

# reverse feature selection
    coef = feature_selection.inverse_transform(coef)

# reverse masking
    weight_img = nifti_masker.inverse_transform(coef)
#plot image
#plot_stat_map(weight_img, average_ana, title='SVM weights')



# ---STEP 6---
### Permunation testing
to measure probablility of chance first score is mean accuracy of the null scores without perm targets, then the array, then the p-value. 



In [39]:
def permutationTesting():
    #null_cv_scores = permutation_test_score(svc, X, y, cv=10)  
    #print(null_cv_scores)

    null_cv_scoresdumb = cross_val_score(DummyClassifier(), X, y, cv=10)  
    print(null_cv_scoresdumb)
    meannull_cv_scoresdumb = np.mean(null_cv_scoresdumb)
    print(meannull_cv_scoresdumb)

In [71]:
def main():
    
############################################
# STEP 1
############################################
    all_func = getData()
    print("GLOBAL BASEPATH: ", basepath)
    print(all_func)
    
    convertToNifti2()

############################################
# STEP 2
############################################    
    y_mask, subs = prepData()
    
############################################
# STEP 3
############################################
    featureSelection(y_mask)
    
############################################
# STEP 4
############################################
    setPrediction()
    
    
############################################
# STEP 5
############################################
    flipImage()
    
############################################
# STEP 6
############################################
    permutationTesting()

In [69]:
main()

sizeof_hdr should be 540; set sizeof_hdr to 540
sizeof_hdr should be 540; set sizeof_hdr to 540


GLOBAL BASEPATH:  /Users/nikkibytes/Documents/NIBL/my_scripts/nilearn_scripts
['/Users/nikkibytes/Documents/NIBL/my_scripts/nilearn_scripts/cs001++.feat/filtered_func_data.nii.gz', '/Users/nikkibytes/Documents/NIBL/my_scripts/nilearn_scripts/cs002++.feat/filtered_func_data.nii.gz']
